# Project 2 - Coronavirus

This project consist to estimate the SIR model with data from WHO.

In [1]:
import pandas as pd
import numpy as np
from scipy import integrate
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, median_absolute_error
from scipy.integrate import odeint
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt

In [2]:
import PDEparams as pde

/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:240: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if error is 'rmse' or error is 'rmsle':
/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:240: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if error is 'rmse' or error is 'rmsle':
/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:245: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if error is 'mse' or error is 'rmse':
/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:245: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if error is 'mse' or error is 'rmse':
/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:247: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif error is 'msle' or error is 'rmsle':
/Users/Octavalo/Documents/GitHub/proyecto2ML/PDEparams.py:247: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif error is 'msle' or error is 'rmsle':
/Users/Octavalo/Documents/GitHub/proyect

## Data from World Health Organization
#### Only laboratory-confirmed, exclude clinically diagnose

In [3]:
data = pd.read_csv('CoV2019.csv')
china = data["China"][:27]#data["China"][:27]
days = data["Days"][:27]
total = data["Total"][:27]
deaths_china = data["Death China"][:27]
other = data["Other"]
china_total = data["China"]
days_total = data["Days"]
deaths_china_total = data["Death China"]
deaths_outside_total = data["Death Outside"]

### Ploting raw data

In [4]:
plt.figure(figsize=(12, 8))
plt.title("Devolopment of COVID-19 until March 1, 2020")
plt.plot(days_total, china_total, '-o', label="China")
plt.plot(days_total, other, '-o', label="Other countries")
plt.plot(days_total, deaths_china_total, '-o', label="Deaths in China")
plt.plot(days_total, deaths_outside_total, '-o', label="Deaths Outside China")
plt.yscale('log')
plt.ylabel("Cases")
plt.xlabel("Days after first WHO report")
plt.legend()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 864x576 with 1 Axes>

### Defining the model

We use a SIR model:

$$\begin{align}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\beta\, \frac{SI}{N}\\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \beta\, \frac{SI}{N} - \gamma\,I\\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma\,I
\end{align}$$

Susceptible -> Infected -> Recovered

$$\begin{align}
\beta &= \text{Contact Rate } \times \text{ Probability of Transmission}\\
\sigma &= \text{Incubation Rate}\\
\gamma &= \text{Recovery Rate}
\end{align}$$

Incubation Period: 1-14 Days, most commonly 5 days (WHO)

### Initial Conditions

In [5]:
Hubei = 5917*10**4
Guangdong = 11346*10**4
Henan = 9605*10**4
Zhejiang = 5737*10**4
Hunan = 6899*10**4
Anhui = 6324*10**4
Jiangxi = 4648*10**4
N = 56*10**3                        # estimate of people affected by lock down
init_I = 1
init_R = 1